In [1]:
!python -V

Python 3.10.0


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715199477344

In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [20]:
with mlflow.start_run():

    mlflow.set_tag("developer", "husni")

    mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [21]:
# eXtreme gradient boosting library
import xgboost as xgb

# distributed hyperparameters optimization library
# fmin is to minimize object function
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [24]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("developer", "husni")
        mlflow.set_tag("model", "xgboost")
        mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.parquet")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

In [25]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[13:15:25] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:12.65486                          
[1]	validation-rmse:8.97999                           
[2]	validation-rmse:7.55846                           
[3]	validation-rmse:7.02632                           
[4]	validation-rmse:6.82784                           
[5]	validation-rmse:6.74403                           
[6]	validation-rmse:6.70531                           
[7]	validation-rmse:6.68212                           
[8]	validation-rmse:6.67250                           
[9]	validation-rmse:6.66504                           
[10]	validation-rmse:6.65746                          
[11]	validation-rmse:6.65575                          
[12]	validation-rmse:6.65172                          
[13]	validation-rmse:6.64915                          
[14]	validation-rmse:6

KeyboardInterrupt: 

In [27]:
with mlflow.start_run():
    mlflow.xgboost.autolog()
    best_params= {
        "learning_rate": 0.2715206941118071,
        "max_depth": 16,
        "min_child_weight": 1.2992126469348504,
        "objective": "reg:linear",
        "reg_alpha": 0.09254548761496846,
        "reg_lambda":	0.022956510190731118,
        "seed": 42
    }
    
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

[13:40:32] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.42002
[1]	validation-rmse:13.09617
[2]	validation-rmse:10.82970
[3]	validation-rmse:9.32524
[4]	validation-rmse:8.34728
[5]	validation-rmse:7.72496
[6]	validation-rmse:7.32802
[7]	validation-rmse:7.07551
[8]	validation-rmse:6.91308
[9]	validation-rmse:6.80539
[10]	validation-rmse:6.73492
[11]	validation-rmse:6.68359
[12]	validation-rmse:6.65023
[13]	validation-rmse:6.62380
[14]	validation-rmse:6.60577
[15]	validation-rmse:6.59443
[16]	validation-rmse:6.58664
[17]	validation-rmse:6.58126
[18]	validation-rmse:6.57735
[19]	validation-rmse:6.57336
[20]	validation-rmse:6.56692
[21]	validation-rmse:6.56410
[22]	validation-rmse:6.56181
[23]	validation-rmse:6.55846
[24]	validation-rmse:6.55555
[25]	validation-rmse:6.55367
[26]	validation-rmse:6.55155
[27]	validation-

2022/05/26 13:41:35 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.


In [28]:
mlflow.xgboost.autolog(disable=True)

In [31]:
with mlflow.start_run():
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    
    best_params= {
        "learning_rate": 0.2715206941118071,
        "max_depth": 16,
        "min_child_weight": 1.2992126469348504,
        "objective": "reg:linear",
        "reg_alpha": 0.09254548761496846,
        "reg_lambda":	0.022956510190731118,
        "seed": 42
    }
    mlflow.log_params(best_params)
    
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # Dump the preprocessor and log it in mlflow
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    
    # Dump and log easily with mlflow
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[14:33:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.42002
[1]	validation-rmse:13.09617
[2]	validation-rmse:10.82970
[3]	validation-rmse:9.32524
[4]	validation-rmse:8.34728
[5]	validation-rmse:7.72496
[6]	validation-rmse:7.32802
[7]	validation-rmse:7.07551
[8]	validation-rmse:6.91308
[9]	validation-rmse:6.80539
[10]	validation-rmse:6.73492
[11]	validation-rmse:6.68359
[12]	validation-rmse:6.65023
[13]	validation-rmse:6.62380
[14]	validation-rmse:6.60577
[15]	validation-rmse:6.59443
[16]	validation-rmse:6.58664
[17]	validation-rmse:6.58126
[18]	validation-rmse:6.57735
[19]	validation-rmse:6.57336
[20]	validation-rmse:6.56692
[21]	validation-rmse:6.56410
[22]	validation-rmse:6.56181
[23]	validation-rmse:6.55846
[24]	validation-rmse:6.55555
[25]	validation-rmse:6.55367
[26]	validation-rmse:6.55155
[27]	validation-

In [33]:
# Try to load the saved model
logged_model_uri = 'runs:/bb1a2740440a444a8231d91d5f027e92/models_mlflow'

# Load model as a PyFuncModel.
loaded_model_pyc = mlflow.pyfunc.load_model(logged_model_uri)

# Load models as XGBoost model
loaded_model_xgb = mlflow.xgboost.load_model(logged_model_uri)

[14:39:56] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:56] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [36]:
# See type of both models
print(loaded_model_pyc, "\n",  loaded_model_xgb)

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: bb1a2740440a444a8231d91d5f027e92
 


In [39]:
# Predict with pyc and xgb models
y_pred_xgb = loaded_model_xgb.predict(valid)
y_pred_pyc = loaded_model_pyc.predict(X_val)

# See prediction results
print(y_pred_xgb[:10])
print(y_pred_pyc[:10])

[15.334059   7.2451816 13.436698  24.440208   9.308665  17.171759
 11.594145   8.103646   8.963856  19.621996 ]
[15.334059   7.2451816 13.436698  24.440208   9.308665  17.171759
 11.594145   8.103646   8.963856  19.621996 ]


In [46]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2022/05/26 18:40:47 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
/Users/husni.zuhdi/Documents/Codes/1-Python-Workshop/mlops-zoomcamp/base/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
